In [102]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime 

import warnings
warnings.filterwarnings("ignore")


In [103]:
#Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [104]:
master_sheet = client.open_by_key("1y3rZuCJkkNQoulXL9OzkvFfIPuKZpl2c7dTsGRoyE2g").worksheet("Summary")
index = get_as_dataframe(master_sheet, evaluate_formulas=True, header=1)
index.dropna(how='all', inplace=True)
index.reset_index(drop=True, inplace=True)

In [105]:
# index.head()

In [106]:
# Filter only active projects
active_projects = index[index["Active Flag"] == 1]

In [107]:
# === SQL Server connection details ===
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""


In [108]:
all_frames = []  # collect per-project DataFrames here

for idx, row in active_projects.iterrows():
    project_name = row["Project Name"]
    schema = row["SQL Schema Name"].strip("[]")  # e.g., [dbo] -> dbo

    # print(f"\n🚀 Processing Project: {project_name} | Schema: {schema}")
    try:
        with pyodbc.connect(conn_str) as conn:
            query = f"""
            WITH t AS (
                SELECT 
                    LISStoreCode,
                    PERCENTILE_CONT(0.5) WITHIN GROUP
                        (ORDER BY TRY_CONVERT(float, InLatitude))
                        OVER (PARTITION BY LISStoreCode) AS MedianLatitude,
                    PERCENTILE_CONT(0.5) WITHIN GROUP
                        (ORDER BY TRY_CONVERT(float, InLongitude))
                        OVER (PARTITION BY LISStoreCode) AS MedianLongitude
                FROM [{database}].[{schema}].[ISPStoreAttendance_NormalExport]
                WHERE [LISStoreCode] NOT LIKE '%SP'
                  AND LOWER([LISStoreCode]) NOT LIKE '%test%'
                  AND TRY_CONVERT(float, InLatitude)  IS NOT NULL
                  AND TRY_CONVERT(float, InLongitude) IS NOT NULL
            )
            SELECT DISTINCT LISStoreCode, MedianLatitude, MedianLongitude
            FROM t;
            """
            df = pd.read_sql(query, conn)

        df["SchemaName"] = schema  # optional: keep track of source
        all_frames.append(df)
        print(f"✔️ {schema}: {len(df)} rows")
    except Exception as e:
        print(f"❌ Error processing {project_name} ({schema}): {e}")

# Combine after the loop
if all_frames:
    final_df = pd.concat(all_frames, ignore_index=True)
    print("\nPreview:")
    print(final_df.head())
else:
    print("No data gathered.")


✔️ AdaniWilmerManagement: 5346 rows
✔️ CholayilManagement: 2062 rows
✔️ DaburPharmaManagement: 16245 rows
✔️ DanoneGTManagement: 14307 rows
✔️ DelMonteManagement: 718 rows
✔️ GCPL_SAMTManagement: 1038 rows
✔️ GCPLFoodsManagement: 1002 rows
✔️ HaleonManagement: 5251 rows
✔️ HimalayaTurmericGTManagement: 160 rows
✔️ Jayanti_GroupManagement: 332 rows
✔️ JyothyLabsManagement: 618 rows
✔️ CamlinManagement: 590 rows
✔️ MarsManagement: 9942 rows
✔️ McnroeManagement: 1514 rows
✔️ PatanjaliManagement: 591 rows
✔️ PidiliteManagement: 2244 rows
✔️ PiramalManagement: 197 rows
✔️ BeamSuntoryManagement: 10601 rows
✔️ ZydusGTManagement: 6360 rows

Preview:
   LISStoreCode  MedianLatitude  MedianLongitude             SchemaName
0  AD-Store-005       22.719771        75.907401  AdaniWilmerManagement
1  AD-Store-014       22.717900        75.908289  AdaniWilmerManagement
2  AD-Store-015       22.741379        75.878514  AdaniWilmerManagement
3  AD-Store-016       22.735120        75.887849  AdaniWilmerM

In [109]:
all_frames2 = []  # collect per-project DataFrames here

for idx, row in active_projects.iterrows():
    project_name = row["Project Name"]
    schema = row["SQL Schema Name"].strip("[]")  # e.g., [dbo] -> dbo

    # print(f"\n🚀 Processing Project: {project_name} | Schema: {schema}")
    try:
        with pyodbc.connect(conn_str) as conn:
            query = f"""
            SELECT * FROM [{database}].[{schema}].StoreMaster WHERE LOWER([LISStoreCode]) NOT LIKE '%test%';
            """
            df = pd.read_sql(query, conn)

        df["SchemaName"] = schema  # optional: keep track of source
        all_frames2.append(df)
        print(f"✔️ {schema}: {len(df)} rows")
    except Exception as e:
        print(f"❌ Error processing {project_name} ({schema}): {e}")

# Combine after the loop
if all_frames:
    StoreMaster_df = pd.concat(all_frames2, ignore_index=True)
    # print("\nPreview:")
    # print(StoreMaster_df.head())
else:
    print("No data gathered.")


✔️ AdaniWilmerManagement: 5710 rows
✔️ CholayilManagement: 3368 rows
✔️ DaburPharmaManagement: 13506 rows
✔️ DanoneGTManagement: 14701 rows
✔️ DelMonteManagement: 59948 rows
✔️ GCPL_SAMTManagement: 1161 rows
✔️ GCPLFoodsManagement: 104115 rows
✔️ HaleonManagement: 6791 rows
✔️ HimalayaTurmericGTManagement: 204 rows
✔️ Jayanti_GroupManagement: 637 rows
✔️ JyothyLabsManagement: 977 rows
✔️ CamlinManagement: 680 rows
✔️ MarsManagement: 10644 rows
✔️ McnroeManagement: 2304 rows
✔️ PatanjaliManagement: 828 rows
✔️ PidiliteManagement: 2980 rows
✔️ PiramalManagement: 314 rows
✔️ BeamSuntoryManagement: 15324 rows
✔️ ZydusGTManagement: 10837 rows


In [110]:
final_storemaster= pd.merge(StoreMaster_df,final_df,on = ['LISStoreCode','SchemaName'], how = 'left')

In [111]:
final_storemaster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255029 entries, 0 to 255028
Data columns (total 34 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   StoreMasterId    255029 non-null  int64  
 1   ISPStoreId       255029 non-null  int64  
 2   ClientStoreCode  153769 non-null  object 
 3   LISStoreCode     255029 non-null  object 
 4   StoreName        255029 non-null  object 
 5   DistributorCode  34806 non-null   object 
 6   DistributorName  35460 non-null   object 
 7   KYCStatus        138 non-null     object 
 8   Division         6783 non-null    object 
 9   ChainCode        255029 non-null  object 
 10  SubChainName     33555 non-null   object 
 11  StoreType        56700 non-null   object 
 12  Format           255029 non-null  object 
 13  StoreClub        93 non-null      object 
 14  StoreClass       10748 non-null   object 
 15  StoreAddress     182099 non-null  object 
 16  StoreStatus      54939 non-null   obje

In [112]:
# final_storemaster.head()

In [113]:
# final_storemaster[final_storemaster['LISStoreCode']=='AD-Store-7508']

In [114]:
# final_storemasterx = final_storemaster[final_storemaster['SchemaName_x']=='AdaniWilmerManagement']

In [115]:
final_storemaster.to_csv('final_storemaster.csv')

In [116]:
# final_df.to_csv('final_df_slatlong.csv')

In [117]:
# StoreMaster_df.to_csv('StoreMaster_df.csv')